# Setup

In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [3]:
# unzip data
!unzip -q "/images.zip" -d images

unzip:  cannot find or open /images.zip, /images.zip.zip or /images.zip.ZIP.


In [3]:
import os
import pickle
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import cv2
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm

# Imports

In [5]:
with open("/Volumes/Agora_Sim/Python/CAA_Proj_1/Lacuna-solar/Train_v03.pkl", "rb") as f:
    obj = pickle.load(f)
    print(type(obj))  # See what type of object is stored
    print(obj)        # Print some content (if not too large)


<class 'pandas.core.frame.DataFrame'>
                     ID boil_nbr    pan_nbr  \
0               ID00rw8       []        [2]   
1            ID014O6EC7       []        [1]   
2             ID020cu0z       []        [1]   
3     ID024YTBkLvRpQahT       []  [1, 2, 2]   
4         ID02vByTw8Htl       []        [1]   
...                 ...      ...        ...   
3307         IDzt0z6lQC      [1]         []   
3308        IDzwqGWYJzj       []        [2]   
3309    IDzxHMiMmh9r04J       []        [2]   
3310           IDzxXgqh       []        [1]   
3311  IDzxqZRHQqVMgRXYQ       []        [2]   

                                           boil_polygon  \
0                                                    []   
1                                                    []   
2                                                    []   
3                                                    []   
4                                                    []   
...                                        

In [2]:
# Read in the training dataset
train = pd.read_csv(f"Train.csv")

### Understanding the dataset

The dataset is split into 3,312 training images and 1,107 test images. In addition to the polygon annotations, each image is enriched with metadata:

img_origin:
- "S" indicates a satellite image.
- "D" indicates a drone image.
**placement**: Provides context on installation environments:
- roof: Installations on rooftops or terraces.
- openspace: Installations in open courtyards or gardens.
- r_openspace: Installations that span both rooftops and outdoor spaces.
- S-unknown: For satellite images where the specific placement could not be determined.



In [3]:
train.head(2)

,ID,img_origin,placement,boil_nbr,pan_nbr,polygon
0,ID00rw8,D,roof,0,2,"[(2087, 2179.0), (2181, 2191.0), (2171, 2223.0..."
1,ID014O6EC7,D,roof,0,1,"[(1327, 1574.0), (1595, 1308.0), (2169, 1744.0..."


In [4]:
# Create a placement mapper
placement_mapper = train[["ID", "placement"]].drop_duplicates().set_index("ID").to_dict()
# Create a "img_origin" mapper
img_origin_mapper = train[["ID", "img_origin"]].drop_duplicates().set_index("ID").to_dict()

# Group by "ID" and sum up boil_nb, pan_nbr
train_df = train.groupby("ID").sum().reset_index()[["ID", "boil_nbr", "pan_nbr"]]

# Map img_origin and placement
train_df["img_origin"] = train_df["ID"].map(img_origin_mapper["img_origin"])
train_df["placement"] = train_df["ID"].map(placement_mapper["placement"])

# Create path column
train_df["path"] = "images/" + train_df["ID"] + ".jpg"

In [5]:
train_df.head(2)

,ID,boil_nbr,pan_nbr,img_origin,placement,path
0,ID00rw8,0,2,D,roof,images/ID00rw8.jpg
1,ID014O6EC7,0,1,D,roof,images/ID014O6EC7.jpg


In [6]:
# Stratified KFold based on multi-label targets
train_df["stratify_label"] = train_df[["boil_nbr", "pan_nbr"]].sum(axis=1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_df["fold"] = -1
for fold, (_, valid_idx) in enumerate(skf.split(train_df, train_df["stratify_label"])):
    train_df.loc[valid_idx, "fold"] = fold

/Users/joaop.cardoso/miniconda3/envs/fcd/lib/python3.12/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [7]:
# Define Transformations
train_transforms = A.Compose([
    A.Resize(384, 384),
    A.HorizontalFlip(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(384, 384),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [8]:
# Custom Dataset
class SolarPanelDataset(Dataset):
    def __init__(self, dataframe, transform=None, to_train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.to_train = to_train

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = cv2.imread(row["path"])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=image)["image"]

        if self.to_train:
          target = torch.tensor([row["boil_nbr"], row["pan_nbr"]], dtype=torch.float32)
          return image, target
        else:
          return image

In [9]:
# Select fold (cross-validation setup)
fold = 0  # Change fold index as needed

# Split data into training and validation sets
train_data = train_df[train_df["fold"] != fold].reset_index(drop=True)
valid_data = train_df[train_df["fold"] == fold].reset_index(drop=True)

# Create dataset objects
dataset_train = SolarPanelDataset(train_data, transform=train_transforms)
dataset_valid = SolarPanelDataset(valid_data, transform=test_transforms)

# Create data loaders
train_loader = DataLoader(
    dataset_train,
    batch_size=8,
    shuffle=True,       # ✅ Shuffles training data
    num_workers=0,      # ✅ Use 0 in Jupyter notebooks to prevent freezing
    pin_memory=False,    # ✅ Speeds up GPU training (remove if using CPU)
    drop_last=True      # ✅ Prevents batch size mismatches
)

valid_loader = DataLoader(
    dataset_valid,
    batch_size=8,
    shuffle=False,  # 🚨 Do NOT shuffle validation set
    num_workers=0,  # ✅ Use 0 in Jupyter notebooks
    pin_memory=False
)


In [10]:
# Model Definition
class EfficientNetRegressor(nn.Module):
    def __init__(self):
        super(EfficientNetRegressor, self).__init__()
        self.model = timm.create_model("tf_efficientnet_b1", pretrained=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 2)

    def forward(self, x):
        return self.model(x)

# Create the model on CPU explicitly
device = torch.device("cpu")  # Ensure CPU mode
model = EfficientNetRegressor().to(device)


In [11]:
# 🔹 Detect if GPU is available, otherwise fallback to CPU
device = torch.device("cpu")
print(f"Using device: {device}")

model = EfficientNetRegressor().to(device)

# 🔹 Loss function (Mean Absolute Error)
criterion = nn.L1Loss()

# 🔹 Optimizer (Adam with a learning rate of 1e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 🔹 Learning rate scheduler (OPTIONAL: decreases LR after every 10 epochs)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 🔹 Model Saving Path
best_model_path = "best_model.pth"


Using device: cpu


In [13]:
# Training Loop
num_epochs = 3
best_loss = float("inf")

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, targets = images.to(device), targets.to(device)  # Move to CPU

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    # Validation Loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            images, targets = images.to(device), targets.to(device)  # Move to CPU
            outputs = model(images)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

    val_loss /= len(valid_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

    # Save Best Model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

Epoch 1/3 - Validation: 100%|██████████| 83/83 [02:59<00:00,  2.17s/it]


Epoch 1/3, Train Loss: 2.4362, Val Loss: 2.3150


Epoch 2/3 - Validation: 100%|██████████| 83/83 [03:02<00:00,  2.20s/it]


Epoch 2/3, Train Loss: 2.1582, Val Loss: 2.2514


Epoch 3/3 - Validation: 100%|██████████| 83/83 [03:06<00:00,  2.24s/it]


Epoch 3/3, Train Loss: 1.9635, Val Loss: 2.1808


In [15]:
# Load Best Model
model.load_state_dict(torch.load(best_model_path))
model.eval()

# Predict on Validation Set
preds = []
true_vals = []
with torch.no_grad():
    for images, targets in tqdm(valid_loader, desc="Predicting on Validation Set"):
        images = images.to(device)
        outputs = model(images).cpu().numpy()
        preds.append(outputs)
        true_vals.append(targets.numpy())
preds = np.concatenate(preds, axis=0)
true_vals = np.concatenate(true_vals, axis=0)

Predicting on Validation Set: 100%|██████████| 83/83 [03:45<00:00,  2.71s/it]


In [16]:
from sklearn.metrics import mean_absolute_error

# Evaluate using MAE
mae = mean_absolute_error(true_vals, preds)
print(f"Validation MAE: {mae:.4f}")

Validation MAE: 2.1836


In [18]:
# Predict on Test Set
test_df = pd.read_csv(f"Test.csv")

test_df["path"] = "images/" + test_df["ID"] + ".jpg"

dataset_test = SolarPanelDataset(test_df, transform=test_transforms, to_train=False)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=False)

test_preds = []
with torch.no_grad():
    for images in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.to(device)
        outputs = model(images).cpu().numpy()
        test_preds.append(outputs)
test_preds = np.concatenate(test_preds, axis=0)

Predicting on Test Set: 100%|██████████| 35/35 [06:03<00:00, 10.38s/it]


In [ ]:
# Create Sample Submission
submission = pd.DataFrame()
submission["ID"] = np.repeat(test_df["ID"].values, 2)
submission["ID"] = submission["ID"] + np.tile(["_boil", "_pan"], len(test_df))
submission["Target"] = test_preds.flatten().clip(0,1000)

# Save Submission
submission.to_csv("SampleSubmission.csv", index=False)
print("Sample submission saved!")

Sample submission saved!
